In [1]:
# Core libs
import pandas as pd
import numpy as np

# Graphic libs
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
pio.renderers.default = "browser"

# sklearn 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# IA + blockchain = profit
import tensorflow as tf
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Lambda, Dropout, BatchNormalization, Activation, Dense, LeakyReLU
from keras.layers.merging import Add, Concatenate
from keras.utils import plot_model, to_categorical

2022-10-23 09:17:54.594640: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 09:17:54.716691: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 09:17:55.328272: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-23 09:17:55.328355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# Helpers

In [2]:
def plot_pairs(components_df, pca):
    fig = px.scatter_matrix(components_df,
        dimensions=['PC '+ str(i+1) + ' ' + '({:.2f}%)'.format(pca.explained_variance_ratio_[i]*100) for i in range(8)],
        color="Valorcomercial",
        title="Scatter matrix of data set",
        labels=components_df.columns) # remove underscore
    fig.update_traces(diagonal_visible=False)
    fig.update_traces(marker_size=1)
    fig.show()

# Training

## Loading data

In [3]:
trainer_data = pd.read_excel('../data/final/LimaPROV.xlsx')
trainer_data = trainer_data.drop(['Provincia', 'Distrito'], axis = 1).replace('Centro de Salud',12).replace('Exterior','1').replace('Interior','2').replace('Malo','6').applymap(lambda x : float(x.replace(',','')) if isinstance(x,str) else x)
trainer_data = trainer_data.applymap(lambda x : np.log10(x + 0.0000001))

#log_data = trainer_data.drop(['Valorcomercial'], axis=1)

In [4]:
#data_pca = PCA(n_components=3)
data_pca = PCA()



data_pca.fit(trainer_data)

X = trainer_data.to_numpy()
X = X.T

components = data_pca.components_@X
components = components.T

#components = data_pca.transform(trainer_data)


In [5]:
components_df = pd.DataFrame(components, columns=['PC '+ str(i+1) + ' ' + '({:.2f}%)'.format(data_pca.explained_variance_ratio_[i]*100) for i in range(13)])
components_df.insert(0, 'Valorcomercial', trainer_data['Valorcomercial'].to_numpy())

plot_pairs(components_df, data_pca)

/home/ric/Documents/Code/hackathonBBVA/venv/lib/python3.10/site-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



In [6]:
## Custom metric
def mape_accuracy(y_true, y_pred):
    correct = tf.reduce_sum(tf.cast( tf.abs((y_true - y_pred)/y_pred) < 0.1, tf.int32), axis=1)
    total = y_true.shape[1]

    return (correct/total)*100

def mape_accuracy_log10(y_true, y_pred):
    correct = tf.reduce_sum(tf.cast(tf.abs(tf.divide(tf.subtract(tf.pow(tf.constant(10, dtype=tf.float32),y_true),tf.pow(tf.constant(10, dtype=tf.float32),y_pred)),tf.pow(tf.constant(10, dtype=tf.float32),y_true))) < 0.1, tf.int32), axis=1)
    #correct = tf.reduce_sum(tf.cast(tf.abs(tf.divide(tf.subtract((y_true + out_center)*out_scale ,(y_pred + out_center)*out_scale),y_true)) < 0.1, tf.int32), axis=1)
    total = y_true.shape[1]

    return (correct/total)*100

# Training data (optimizer)

In [19]:
Xin = Input(shape=(12,),name='In')

X = Dense(7,activation=tf.keras.activations.softsign)(Xin)
X = Dense(7,activation=tf.keras.activations.softsign)(X)
X = Dense(7,activation=tf.keras.activations.softsign)(X)
X = Dense(7,activation=tf.keras.activations.softsign)(X)

completion = Dense(1, activation='relu')(X)

full_pca_inference = Concatenate(axis=1, name = 'completion')([Xin,completion])

Yout = Lambda(lambda x : tf.transpose(data_pca.components_@tf.transpose(x)))(full_pca_inference)

PCA_model = Model(inputs = Xin, outputs = [Yout], name = 'PCA')


Yfinal = Lambda(lambda x : tf.pow(tf.constant(10, dtype=tf.float32), x))(Yout)

S4D_Her0 = Model(inputs = Xin, outputs = [completion], name = 'predictor')

In [20]:
# PCA_model.load_weights('S4D_Her0_W/PCA/PCA_trained')
# 
# PCA_model.compile(loss      = 'mae',
                #   metrics   = ['accuracy'])
# 
# scores = PCA_model.evaluate(log_data, components, verbose=1)

In [21]:
epochs     = 100       # número de epocas
batch_size = 10       # tamaño del lote
alpha      = 0.0005    # razon de aprendizaje
decay      = 0.00001    # decaimiento de alpha

PCA_model.compile(optimizer = optimizers.Adam(learning_rate=alpha, decay=decay),
                  loss      = 'mae',
                  metrics   = [mape_accuracy_log10])

history = PCA_model.fit(x                = log_data,
                        y                = components,
                        batch_size       = batch_size,
                        epochs           = epochs,
                        validation_split = 0.2,
                        verbose          = 1)

Epoch 1/100
140/140 [==============================] - 1s 3ms/step - loss: 0.6731 - mape_accuracy_log10: 30.6318 - val_loss: 0.5670 - val_mape_accuracy_log10: 30.7692
Epoch 2/100
140/140 [==============================] - 0s 3ms/step - loss: 0.4720 - mape_accuracy_log10: 30.9065 - val_loss: 0.3791 - val_mape_accuracy_log10: 31.4945
Epoch 3/100
140/140 [==============================] - 0s 3ms/step - loss: 0.3104 - mape_accuracy_log10: 35.1319 - val_loss: 0.2370 - val_mape_accuracy_log10: 39.1209
Epoch 4/100
140/140 [==============================] - 0s 3ms/step - loss: 0.1810 - mape_accuracy_log10: 45.4725 - val_loss: 0.1205 - val_mape_accuracy_log10: 55.0769
Epoch 5/100
140/140 [==============================] - 0s 3ms/step - loss: 0.0824 - mape_accuracy_log10: 66.3571 - val_loss: 0.0569 - val_mape_accuracy_log10: 75.3626
Epoch 6/100
140/140 [==============================] - 0s 3ms/step - loss: 0.0549 - mape_accuracy_log10: 74.6429 - val_loss: 0.0559 - val_mape_accuracy_log10: 73.978

In [26]:
S4D_Her0.compile(loss      = tf.losses.MeanAbsolutePercentageError(),
                 metrics   = [mape_accuracy])

score = S4D_Her0.evaluate(log_data, trainer_data['Valorcomercial'].to_numpy(), verbose=1)

55/55 [==============================] - 0s 2ms/step - loss: 4.0937 - mape_accuracy: 90.1143


In [16]:
X_emmbedder_train = components.copy()
Y_emmbedder_train = trainer_data['Valorcomercial'].to_numpy()

In [22]:
Xin = Input(shape=(13,),name='transformed_input')

X = Dense(50, activation=None, name='hidden_2_d')(Xin)
X = LeakyReLU(0.5, name='hidden_2_a')(X)

X = Dropout(0.2)(X)

X = Dense(50, activation=None, name='hidden_4_d')(X)
X = LeakyReLU(0.5, name='hidden_4_a')(X)

X = Dropout(0.2)(X)

X = Dense(20, activation=None, name='hidden_12_d')(X)
X = LeakyReLU(0.3, name='hidden_12_a')(X)

X = Dense(1, activation=None, name='predictor_d')(X)

Yout = LeakyReLU(0.2, name='predictor_a')(X)


S4D_jengi = Model(inputs = Xin, outputs = [Yout], name = 'S4DJengi')

In [23]:
# S4D_jengi.load_weights('S4D_Her0_W/Jengi/Jengi_trained')
# 
# S4D_jengi.compile(loss      = 'mae',
                #   metrics   = [mape_accuracy_log10])
# 
# S4D_jengi.evaluate(X_emmbedder_train, Y_emmbedder_train, verbose=1)

In [31]:
epochs     = 1000       # número de epocas
batch_size = 200       # tamaño del lote
alpha      = 0.0001    # razon de aprendizaje
decay      = 0.00001    # decaimiento de alpha

S4D_jengi.compile(optimizer = optimizers.Adam(learning_rate=alpha, decay=decay),
                  loss      = 'mae',
                  metrics   = [mape_accuracy_log10])


history = S4D_jengi.fit(x                = X_emmbedder_train,
                        y                = Y_emmbedder_train,
                        batch_size       = batch_size,
                        epochs           = epochs,
                        validation_split = 0.5,
                        verbose          = 1)


Epoch 1/1000
5/5 [==============================] - 1s 38ms/step - loss: 0.0289 - mape_accuracy_log10: 82.1714 - val_loss: 0.0498 - val_mape_accuracy_log10: 64.2857
Epoch 2/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.0289 - mape_accuracy_log10: 82.1714 - val_loss: 0.0510 - val_mape_accuracy_log10: 64.5714
Epoch 3/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.0287 - mape_accuracy_log10: 81.7143 - val_loss: 0.0508 - val_mape_accuracy_log10: 64.5714
Epoch 4/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.0310 - mape_accuracy_log10: 80.5714 - val_loss: 0.0508 - val_mape_accuracy_log10: 64.5714
Epoch 5/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.0280 - mape_accuracy_log10: 82.5143 - val_loss: 0.0502 - val_mape_accuracy_log10: 64.5714
Epoch 6/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.0282 - mape_accuracy_log10: 82.8571 - val_loss: 0.0490 - val_mape_accuracy_log10: 64.5714
Epoch 7/10

KeyboardInterrupt: 

In [55]:
#S4D_jengi.save_weights('S4D_Her0_W/Jengi/Jengi_trained')

In [35]:
X_inverse_train = np.concatenate((np.expand_dims(Y_emmbedder_train,axis=1), log_data), axis=1)
Y_inverse_train = np.power(10,Y_emmbedder_train)

In [36]:
Xin = Input(shape=(13,),name='transformed_input')
X = Dense(15, activation='relu', name='hidden_1')(Xin)
X = Dense(100, activation=tf.keras.activations.exponential, name='hidden_2')(X)
X = Dense(100, activation='relu', name='hidden_3')(X)
X = Dense(50, activation='relu', name='hidden_4')(X)
X = Dense(25, activation='relu', name='hidden_5')(X)
X = Dense(10, activation='relu', name='hidden_6')(X)
X = Dense(5, activation='relu', name='hidden_7')(X)
X = Dense(5, activation='relu', name='hidden_8')(X)
Yout = Dense(1, activation=None, name='predictor')(X)

S4D_gumbo = Model(inputs = Xin, outputs = [Yout], name = 'S4DGumbo')

In [34]:
# S4D_gumbo.load_weights('S4D_Her0_W/Gumbo/Gumbo_trained')
# 
# S4D_gumbo.compile(loss      = tf.keras.losses.mean_absolute_percentage_error,
                #   metrics   = [mape_accuracy])
# 
# S4D_gumbo.evaluate(X_inverse_train, Y_inverse_train, verbose=1)

55/55 [==============================] - 0s 2ms/step - loss: 90.5798 - mape_accuracy: 0.0000e+00


[90.57975006103516, 0.0]

In [42]:
epochs     = 1000       # número de epocas
batch_size = 100        # tamaño del lote
alpha      = 0.001    # razon de aprendizaje
decay      = 0.00001    # decaimiento de alpha

S4D_gumbo.compile(optimizer = optimizers.Adam(learning_rate=alpha, decay=decay),
                  loss      = tf.keras.losses.mean_absolute_percentage_error,
                  metrics   = [mape_accuracy])

history = S4D_gumbo.fit(x                = X_inverse_train,
                        y                = Y_inverse_train,
                        batch_size       = batch_size,
                        epochs           = epochs,
                        validation_split = 0.2,
                        verbose          = 1)

Epoch 1/1000
14/14 [==============================] - 1s 16ms/step - loss: 6.8522 - mape_accuracy: 76.4286 - val_loss: 9.6122 - val_mape_accuracy: 96.8571
Epoch 2/1000
14/14 [==============================] - 0s 6ms/step - loss: 3.1231 - mape_accuracy: 97.4286 - val_loss: 9.9632 - val_mape_accuracy: 98.0000
Epoch 3/1000
14/14 [==============================] - 0s 7ms/step - loss: 2.4788 - mape_accuracy: 99.0000 - val_loss: 10.2049 - val_mape_accuracy: 98.8571
Epoch 4/1000
14/14 [==============================] - 0s 6ms/step - loss: 1.7965 - mape_accuracy: 99.4286 - val_loss: 7.8917 - val_mape_accuracy: 98.2857
Epoch 5/1000
14/14 [==============================] - 0s 5ms/step - loss: 1.8485 - mape_accuracy: 99.4286 - val_loss: 9.0975 - val_mape_accuracy: 98.2857
Epoch 6/1000
14/14 [==============================] - 0s 5ms/step - loss: 2.1255 - mape_accuracy: 99.5000 - val_loss: 6.6756 - val_mape_accuracy: 98.5714
Epoch 7/1000
14/14 [==============================] - 0s 5ms/step - loss: 

KeyboardInterrupt: 

In [56]:
#S4D_gumbo.save_weights('S4D_Her0_W/Gumbo/Gumbo_trained')

In [48]:
#PCA_model.load_weights('S4D_Her0_W/PCA/PCA_trained')
#S4D_jengi.load_weights('S4D_Her0_W/Jengi/Jengi_trained')
#S4D_gumbo.load_weights('S4D_Her0_W/Gumbo/Gumbo_trained')

Xin = Input(shape=(12,),name='logdata')

inf_comp = PCA_model(Xin)

# Jengi
comval_l10 = S4D_jengi(inf_comp)

# Bridge
log_rep = Concatenate(axis=1)([comval_l10, Xin])

# Gumbo
Yout = S4D_gumbo(log_rep)

S4D_Her0 = Model(inputs = Xin, outputs = [Yout], name = 'S4DHer0')

In [93]:
S4D_Her0.compile(loss      = tf.keras.losses.MeanAbsolutePercentageError(),
                 metrics   = [mape_accuracy])

shape = S4D_Her0.predict(log_data)

1332/1332 [==============================] - 3s 2ms/step


In [49]:
for layer in S4D_Her0.layers:
    if layer.name in ['S4DJengi']:
        layer.trainable = False
    else:
        layer.trainable = True

In [54]:
epochs     = 1000       # número de epocas
batch_size = 100       # tamaño del lote
alpha      = 0.00001    # razon de aprendizaje
decay      = 0.00001    # decaimiento de alpha

S4D_Her0.compile(optimizer = optimizers.Adam(learning_rate=alpha, decay=decay),
                  loss      = 'mae',
                  metrics   = [mape_accuracy])

S4D_Her0.fit(x                = log_data,
             y                = Y_inverse_train,
             batch_size       = batch_size,
             epochs           = epochs,
             validation_split = 0.3,
             verbose          = 1)

Epoch 1/1000
13/13 [==============================] - 1s 22ms/step - loss: 130710.9453 - mape_accuracy: 25.7143 - val_loss: 435035712.0000 - val_mape_accuracy: 19.0476
Epoch 2/1000
13/13 [==============================] - 0s 7ms/step - loss: 130143.5000 - mape_accuracy: 26.1224 - val_loss: 432379328.0000 - val_mape_accuracy: 19.4286
Epoch 3/1000
13/13 [==============================] - 0s 6ms/step - loss: 130264.9453 - mape_accuracy: 26.6122 - val_loss: 439527200.0000 - val_mape_accuracy: 19.0476
Epoch 4/1000
13/13 [==============================] - 0s 7ms/step - loss: 131346.7344 - mape_accuracy: 25.6327 - val_loss: 449887584.0000 - val_mape_accuracy: 20.0000
Epoch 5/1000
13/13 [==============================] - 0s 7ms/step - loss: 130259.5156 - mape_accuracy: 25.3061 - val_loss: 445860352.0000 - val_mape_accuracy: 19.6190
Epoch 6/1000
13/13 [==============================] - 0s 10ms/step - loss: 129220.5859 - mape_accuracy: 26.4490 - val_loss: 437443168.0000 - val_mape_accuracy: 19.6

KeyboardInterrupt: 